In [1]:
import os  
import glob
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

import PIL 
import random
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torchinfo import summary 

import torch.optim as optim
from IPython.display import Image
from torch.utils.data import DataLoader, Dataset

from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch.nn.functional as F


import cv2
import pydicom
import matplotlib.pyplot as plt

In [2]:
from duke_dbt_data import dcmread_image

In [3]:
boxes_train = pd.read_csv("C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/BCS-DBT boxes-train-v2.csv")               #Not Use yet
file_path_test = pd.read_csv("C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/BCS-DBT file-paths-test-v2.csv")        #Not Use
file_path_train = pd.read_csv("C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/BCS-DBT file-paths-train-v2.csv")      
labels_train = pd.read_csv("C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/BCS-DBT labels-train-v2.csv")
file_path_valid = pd.read_csv("C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/BCS-DBT file-paths-validation-v2.csv") #Not Use

In [4]:
file_path_test

,PatientID,StudyUID,View,descriptive_path,classic_path
0,DBT-P00036,DBT-S03354,lcc,Breast-Cancer-Screening-DBT/DBT-P00036/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00036/1.2.826...
1,DBT-P00036,DBT-S03354,lmlo,Breast-Cancer-Screening-DBT/DBT-P00036/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00036/1.2.826...
2,DBT-P00036,DBT-S03354,rcc,Breast-Cancer-Screening-DBT/DBT-P00036/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00036/1.2.826...
3,DBT-P00036,DBT-S03354,rmlo,Breast-Cancer-Screening-DBT/DBT-P00036/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00036/1.2.826...
4,DBT-P00087,DBT-S04170,lcc,Breast-Cancer-Screening-DBT/DBT-P00087/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00087/1.2.826...
...,...,...,...,...,...
1716,DBT-P04790,DBT-S04022,rmlo,Breast-Cancer-Screening-DBT/DBT-P04790/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P04790/1.2.826...
1717,DBT-P00129,DBT-S03696,lcc,Breast-Cancer-Screening-DBT/DBT-P00129/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00129/1.2.826...
1718,DBT-P00129,DBT-S03696,lmlo,Breast-Cancer-Screening-DBT/DBT-P00129/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00129/1.2.826...
1719,DBT-P00129,DBT-S03696,rcc,Breast-Cancer-Screening-DBT/DBT-P00129/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00129/1.2.826...


### Plot some images

In [5]:
# df = file_path_test
# view_series = df.iloc[0]
# view = view_series["View"]
# # if you have image data saved in classic path folder structure, read the file path from "classic_path"
# # image_path = os.path.join("D:/LUCAS/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/test/manifest-1617905855234/", view_series["descriptive_path"])
# image_path = "D:/LUCAS/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/test/manifest-1617905855234/Breast-Cancer-Screening-DBT/DBT-P00036/01-01-2000-DBT-S03354-MAMMO screening digital bilateral-14628/12278.000000-NA-02208/1-1.dcm"

In [6]:
# image = dcmread_image(fp=image_path, view=view)
# plt.imshow(image[1], cmap=plt.cm.gray);

In [7]:
# # Load the DICOM image
# dicom_data = pydicom.dcmread(image_path)
# dicom_data

### Some Informations

In [8]:
unique_pat = list(pd.unique(labels_train["PatientID"]))

In [9]:
neg_pat = []
pos_pat = []
for pat in unique_pat:
    flag = list(labels_train.loc[labels_train["PatientID"] == pat]['Cancer'])
    if flag[0]:
        pos_pat.append(pat)
    else:
        neg_pat.append(pat)
print(f"Patient with cancer {len(pos_pat)}")
print(f"Patient with not cancer {len(neg_pat)}")

Patient with cancer 39
Patient with not cancer 4323


### Fix file_path_train

In [10]:
# Path to add in front
base_path = "C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/manifest-1617905855234"

# Function to replace the last hyphen with "-NA-" in the last folder
def replace_last_hyphen(path):
    parts = path.split("/")
    last_folder = parts[-2]
    last_hyphen_index = last_folder.rfind("-")
    if last_hyphen_index != -1:
        parts[-2] = last_folder[:last_hyphen_index] + "-NA-" + last_folder[last_hyphen_index+1:]
    return "/".join(parts)

# Apply the function to the descriptive_path column
file_path_train["descriptive_path"] = file_path_train["descriptive_path"].apply(replace_last_hyphen)

# Concatenate the base path with the updated "descriptive_path"
file_path_train["fullpath"] = base_path + "/" + file_path_train["descriptive_path"]

file_path_train


,PatientID,StudyUID,View,descriptive_path,classic_path,fullpath
0,DBT-P00013,DBT-S00163,rmlo,Breast-Cancer-Screening-DBT/DBT-P00013/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00013/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
1,DBT-P00023,DBT-S04378,lcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
2,DBT-P00023,DBT-S04378,lmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
3,DBT-P00023,DBT-S04378,rcc,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
4,DBT-P00023,DBT-S04378,rmlo,Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P00023/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
...,...,...,...,...,...,...
19143,DBT-P04356,DBT-S00073,rmlo,Breast-Cancer-Screening-DBT/DBT-P04356/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P04356/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
19144,DBT-P01481,DBT-S01753,lcc,Breast-Cancer-Screening-DBT/DBT-P01481/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P01481/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
19145,DBT-P01481,DBT-S01753,lmlo,Breast-Cancer-Screening-DBT/DBT-P01481/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P01481/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...
19146,DBT-P01481,DBT-S01753,rcc,Breast-Cancer-Screening-DBT/DBT-P01481/01-01-2...,Breast-Cancer-Screening-DBT/DBT-P01481/1.2.826...,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...


In [11]:
file_path_train["fullpath"][1]

'C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/manifest-1617905855234/Breast-Cancer-Screening-DBT/DBT-P00023/01-01-2000-DBT-S04378-MAMMO SCREENING DIGITAL BILATERAL-20650/19711.000000-NA-14072/1-1.dcm'

### Split Data

In [12]:
merged_df = pd.merge(file_path_train, labels_train, on="PatientID")
merged_df  = merged_df[['PatientID', 'StudyUID_x', 'View_x', 'fullpath', 'StudyUID_y', 'View_y', 'Cancer']]
merged_df

,PatientID,StudyUID_x,View_x,fullpath,StudyUID_y,View_y,Cancer
0,DBT-P00013,DBT-S00163,rmlo,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S00163,rmlo,0
1,DBT-P00023,DBT-S04378,lcc,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S04378,lcc,0
2,DBT-P00023,DBT-S04378,lcc,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S04378,lmlo,0
3,DBT-P00023,DBT-S04378,lcc,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S04378,rcc,0
4,DBT-P00023,DBT-S04378,lcc,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S04378,rmlo,0
...,...,...,...,...,...,...,...
91569,DBT-P01481,DBT-S01753,rcc,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S01753,rmlo,0
91570,DBT-P01481,DBT-S01753,rmlo,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S01753,lcc,0
91571,DBT-P01481,DBT-S01753,rmlo,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S01753,lmlo,0
91572,DBT-P01481,DBT-S01753,rmlo,C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_...,DBT-S01753,rcc,0


In [13]:
len(pos_pat), len(neg_pat)

(39, 4323)

In [14]:
# Divisão dos pacientes em conjuntos de treino, teste e validação
train_ratio = 0.7
test_ratio = 0.15
val_ratio = 0.15

# Embaralhe a ordem dos pacientes
random.shuffle(pos_pat)
random.shuffle(neg_pat)

# Separe os pacientes em conjuntos de treino, teste e valid
train_patients = pos_pat[:int(len(pos_pat) * train_ratio)] + neg_pat[:int(len(neg_pat) * train_ratio)]

test_patients = pos_pat[int(len(pos_pat) * train_ratio):int(len(pos_pat) * (train_ratio + test_ratio))] + \
                neg_pat[int(len(neg_pat) * train_ratio):int(len(neg_pat) * (train_ratio + test_ratio))]
                
valid_patients = pos_pat[int(len(pos_pat) * (train_ratio + test_ratio)):] + \
                 neg_pat[int(len(neg_pat) * (train_ratio + test_ratio)):]

In [15]:
train_df = merged_df[merged_df['PatientID'].isin(train_patients)]
test_df = merged_df[merged_df['PatientID'].isin(test_patients)]
valid_df = merged_df[merged_df['PatientID'].isin(valid_patients)]

In [16]:
train_path, train_label = list(train_df['fullpath']), list(train_df['Cancer'])
test_path, test_label = list(test_df['fullpath']), list(test_df['Cancer'])
valid_path, valid_label = list(valid_df['fullpath']), list(valid_df['Cancer'])

In [17]:
len(train_path), len(train_label)

(63665, 63665)

In [18]:
len(test_path), len(test_label)

(13830, 13830)

In [19]:
len(valid_path), len(valid_label)

(14079, 14079)

In [20]:
print("Número de pacientes no conjunto de treino:", len(train_patients))
print("Número de pacientes no conjunto de teste:", len(test_patients))
print("Número de pacientes no conjunto de validação:", len(valid_patients))

Número de pacientes no conjunto de treino: 3053
Número de pacientes no conjunto de teste: 654
Número de pacientes no conjunto de validação: 655


In [21]:
len(neg_pat) + len(pos_pat) == len(train_patients)+len(test_patients)+len(valid_patients)

True

In [22]:
def encontrar_elementos_comuns(lista1, lista2, lista3):
    # Converte as listas para conjuntos para facilitar a interseção
    conjunto1 = set(lista1)
    conjunto2 = set(lista2)
    conjunto3 = set(lista3)

    # Encontra os elementos que estão presentes em todas as listas
    elementos_comuns = conjunto1.intersection(conjunto2, conjunto3)
    return list(elementos_comuns)

# Exemplo de listas
lista1 = train_path
lista2 = test_path
lista3 = valid_path

# Encontrar elementos comuns
elementos_comuns = encontrar_elementos_comuns(lista1, lista2, lista3)

# Exibir os elementos comuns
print("Elementos comuns nas três listas:", elementos_comuns)


Elementos comuns nas três listas: []


### Saving the lists

In [23]:
# In Notebook 1
import json


with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/train_path.json', 'w') as f:
    json.dump(train_path, f)
    
with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/train_label.json', 'w') as f:
    json.dump(train_label, f)


with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/test_path.json', 'w') as f:
    json.dump(test_path, f)
    
with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/test_label.json', 'w') as f:
    json.dump(test_label, f)
    
    
with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/valid_path.json', 'w') as f:
    json.dump(valid_path, f)
    
with open('C:/Users/Lucas/Documents/PIBIC/DATASET/CANCER_IMAGING_ARCHIVE/lists_preprocessing/valid_label.json', 'w') as f:
    json.dump(valid_label, f)
